<a href="https://colab.research.google.com/github/leman-cap13/my_projects/blob/main/Multi_Task_NER_POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#model

In [175]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (3).json


{'kaggle (3).json': b'{"username":"lmanqasml","key":"2d851a4eb9cae06770577185722326e0"}'}

In [176]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [177]:
!kaggle datasets download rohitr4307/ner-dataset

Dataset URL: https://www.kaggle.com/datasets/rohitr4307/ner-dataset
License(s): unknown
ner-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [178]:
import zipfile

In [179]:
with zipfile.ZipFile('/content/ner-dataset.zip','r') as zip_ref:
  zip_ref.extractall()

In [180]:
import pandas as pd
df=pd.read_csv('/content/NER_Dataset.csv')
df

,Sentence_ID,Word,POS,Tag
0,Sentence: 1,"['Thousands', 'of', 'demonstrators', 'have', '...","['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 10,"['Iranian', 'officials', 'say', 'they', 'expec...","['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
2,Sentence: 100,"['Helicopter', 'gunships', 'Saturday', 'pounde...","['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...","['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '..."
3,Sentence: 1000,"['They', 'left', 'after', 'a', 'tense', 'hour-...","['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 10000,"['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...","['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...","['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-..."
...,...,...,...,...
47954,Sentence: 9995,"['Opposition', 'leader', 'Mir', 'Hossein', 'Mo...","['NNP', 'NN', 'NNP', 'NNP', 'NNP', 'VBZ', 'VBN...","['O', 'O', 'O', 'B-per', 'I-per', 'O', 'O', 'O..."
47955,Sentence: 9996,"['On', 'Thursday', ',', 'Iranian', 'state', 'm...","['IN', 'NNP', ',', 'JJ', 'NN', 'NNS', 'VBN', '...","['O', 'B-tim', 'O', 'B-gpe', 'O', 'O', 'O', 'O..."
47956,Sentence: 9997,"['Following', 'Iran', ""'s"", 'disputed', 'June'...","['VBG', 'NNP', 'POS', 'JJ', 'NNP', 'CD', 'NNS'...","['O', 'B-geo', 'O', 'O', 'B-tim', 'I-tim', 'O'..."
47957,Sentence: 9998,"['Since', 'then', ',', 'authorities', 'have', ...","['IN', 'RB', ',', 'NNS', 'VBP', 'VBN', 'JJ', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [181]:
df.columns

Index(['Sentence_ID', 'Word', 'POS', 'Tag'], dtype='object')

In [182]:
df['Word'].iloc[4]

"['U.N.', 'relief', 'coordinator', 'Jan', 'Egeland', 'said', 'Sunday', ',', 'U.S.', ',', 'Indonesian', 'and', 'Australian', 'military', 'helicopters', 'are', 'ferrying', 'out', 'food', 'and', 'supplies', 'to', 'remote', 'areas', 'of', 'western', 'Aceh', 'province', 'that', 'ground', 'crews', 'can', 'not', 'reach', '.']"

In [183]:
df['Tag'].unique()

array(["['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']",
       "['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O']",
       "['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'I-geo', 'O']",
       ...,
       "['O', 'B-tim', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
       "['O', 'B-geo', 'O', 'O', 'B-tim', 'I-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']",
       "['O', 'B-org', 'I-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'I-tim', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"],
      dt

In [184]:
#B-gpe, B-tim, B-geo, B-org,I-geo, I-org, I-tim ---BIO Formats

In [185]:
import numpy as np

In [186]:
# First Step Tokenization
# our input is already token

In [187]:
# Convert Token to token id

In [188]:
import ast
df['Word'] = df['Word'].apply(ast.literal_eval)


In [189]:
# Pos ucun Eval appyle et
df['POS']=df['POS'].apply(ast.literal_eval)

In [190]:
df['Word'].head()

,Word
0,"[Thousands, of, demonstrators, have, marched, ..."
1,"[Iranian, officials, say, they, expect, to, ge..."
2,"[Helicopter, gunships, Saturday, pounded, mili..."
3,"[They, left, after, a, tense, hour-long, stand..."
4,"[U.N., relief, coordinator, Jan, Egeland, said..."


In [191]:
all_tokens = [token for row in df['Word'] for token in row]
all_tokens

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.',
 'Iranian',
 'officials',
 'say',
 'they',
 'expect',
 'to',
 'get',
 'access',
 'to',
 'sealed',
 'sensitive',
 'parts',
 'of',
 'the',
 'plant',
 'Wednesday',
 ',',
 'after',
 'an',
 'IAEA',
 'surveillance',
 'system',
 'begins',
 'functioning',
 '.',
 'Helicopter',
 'gunships',
 'Saturday',
 'pounded',
 'militant',
 'hideouts',
 'in',
 'the',
 'Orakzai',
 'tribal',
 'region',
 ',',
 'where',
 'many',
 'Taliban',
 'militants',
 'are',
 'believed',
 'to',
 'have',
 'fled',
 'to',
 'avoid',
 'an',
 'earlier',
 'military',
 'offensive',
 'in',
 'nearby',
 'South',
 'Waziristan',
 '.',
 'They',
 'left',
 'after',
 'a',
 'tense',
 'hour-long',
 'standoff',
 'with',
 'riot',
 'police',
 '.',
 'U.N.',
 'relief',
 'coordinator',
 'Jan',
 'Egeland',
 'sai

In [192]:
vocab = {token: idx+1 for idx, token in enumerate(sorted(set(all_tokens)))}

In [193]:
vocab_size=len(vocab)+1

In [194]:
vocab_size

35179

In [195]:
def tokens_to_ids(token_list, vocab):
    return [vocab.get(token, 0) for token in token_list]

token_ids = tokens_to_ids(all_tokens, vocab)


In [196]:
df['token_ids'] = df['Word'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

In [197]:
df['token_ids']

,token_ids
0,"[15078, 27701, 20970, 24219, 26435, 33390, 968..."
1,"[8194, 27728, 31034, 33290, 22578, 33465, 2372..."
2,"[7599, 24040, 13560, 28906, 26766, 24371, 2485..."
3,"[15050, 25893, 16916, 16575, 33180, 24593, 323..."
4,"[15425, 30228, 20255, 8421, 5853, 30958, 14608..."
...,...
47954,"[11663, 25852, 10573, 7809, 10792, 24198, 3095..."
47955,"[11627, 15091, 22, 8194, 32336, 26571, 29457, ..."
47956,"[6522, 8189, 18, 21474, 8681, 316, 22006, 22, ..."
47957,"[14090, 33260, 22, 17933, 24219, 24317, 29446,..."


In [198]:
# tag token id cevirmeliyik

In [199]:
df['Tag'].dtype

dtype('O')

In [200]:
import ast

df['Tag'] = df['Tag'].apply(ast.literal_eval)


In [201]:
#uniques for POS
uniques_pos=sorted(df['POS'].explode().unique())
pos2id={pos: idx for idx,pos in enumerate(uniques_pos)}
id2pos={idx: pos for pos,idx in pos2id.items()}

In [202]:
uniques_tag=sorted(df['Tag'].explode().unique())
uniques_tag

['B-art',
 'B-eve',
 'B-geo',
 'B-gpe',
 'B-nat',
 'B-org',
 'B-per',
 'B-tim',
 'I-art',
 'I-eve',
 'I-geo',
 'I-gpe',
 'I-nat',
 'I-org',
 'I-per',
 'I-tim',
 'O']

In [203]:
tag2id = {tag: idx for idx, tag in enumerate(uniques_tag)}
id2tag = {idx: tag for tag, idx in tag2id.items()}

In [204]:
tag2id

{'B-art': 0,
 'B-eve': 1,
 'B-geo': 2,
 'B-gpe': 3,
 'B-nat': 4,
 'B-org': 5,
 'B-per': 6,
 'B-tim': 7,
 'I-art': 8,
 'I-eve': 9,
 'I-geo': 10,
 'I-gpe': 11,
 'I-nat': 12,
 'I-org': 13,
 'I-per': 14,
 'I-tim': 15,
 'O': 16}

In [205]:
id2tag

{0: 'B-art',
 1: 'B-eve',
 2: 'B-geo',
 3: 'B-gpe',
 4: 'B-nat',
 5: 'B-org',
 6: 'B-per',
 7: 'B-tim',
 8: 'I-art',
 9: 'I-eve',
 10: 'I-geo',
 11: 'I-gpe',
 12: 'I-nat',
 13: 'I-org',
 14: 'I-per',
 15: 'I-tim',
 16: 'O'}

In [206]:
df['tag_ids'] = df['Tag'].apply(lambda tags: [tag2id[tag] for tag in tags])

In [207]:
df['pos_ids']  = df['POS'].apply(lambda tags: [pos2id[tag] for tag in tags])

In [208]:
df['tag_ids']

,tag_ids
0,"[16, 16, 16, 16, 16, 16, 2, 16, 16, 16, 16, 16..."
1,"[3, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16..."
2,"[16, 16, 7, 16, 16, 16, 16, 16, 2, 16, 16, 16,..."
3,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]"
4,"[2, 16, 16, 6, 14, 16, 7, 16, 2, 16, 3, 16, 3,..."
...,...
47954,"[16, 16, 16, 6, 14, 16, 16, 16, 16, 16, 16, 16..."
47955,"[16, 7, 16, 3, 16, 16, 16, 16, 16, 16, 16, 16,..."
47956,"[16, 2, 16, 16, 7, 15, 16, 16, 16, 16, 16, 16,..."
47957,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."


In [209]:
# Indi Token id leri paddin etmek lazimdiki hem imput hem output uzunlugu eyni olsun

In [210]:
import tensorflow as tf

In [211]:
df['seq_len'] = df['token_ids'].apply(len)

print("Max Length:", df['seq_len'].max())

Max Length: 104


In [212]:
MAX_LEN = int(df['seq_len'].quantile(0.95))

In [213]:
MAX_LEN

35

In [215]:
print(pos2id.keys())

dict_keys(['$', ',', '.', ':', ';', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LRB', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'RRB', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``'])


In [224]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 35

df['input_ids'] = list(pad_sequences(df['token_ids'], maxlen=MAX_LEN, padding='post', value=0))
df['label_ids'] = list(pad_sequences(df['tag_ids'], maxlen=MAX_LEN, padding='post', value=tag2id['O']))

# pos ucun padding
df['pos_label_ids']=list(pad_sequences(df['pos_ids'], maxlen=MAX_LEN, padding='post',value=pos2id['NN']))


In [218]:
df['input_ids']

,input_ids
0,"[15078, 27701, 20970, 24219, 26435, 33390, 968..."
1,"[8194, 27728, 31034, 33290, 22578, 33465, 2372..."
2,"[7599, 24040, 13560, 28906, 26766, 24371, 2485..."
3,"[15050, 25893, 16916, 16575, 33180, 24593, 323..."
4,"[15425, 30228, 20255, 8421, 5853, 30958, 14608..."
...,...
47954,"[11663, 25852, 10573, 7809, 10792, 24198, 3095..."
47955,"[11627, 15091, 22, 8194, 32336, 26571, 29457, ..."
47956,"[6522, 8189, 18, 21474, 8681, 316, 22006, 22, ..."
47957,"[14090, 33260, 22, 17933, 24219, 24317, 29446,..."


In [219]:
df['label_ids']

,label_ids
0,"[16, 16, 16, 16, 16, 16, 2, 16, 16, 16, 16, 16..."
1,"[3, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16..."
2,"[16, 16, 7, 16, 16, 16, 16, 16, 2, 16, 16, 16,..."
3,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."
4,"[2, 16, 16, 6, 14, 16, 7, 16, 2, 16, 3, 16, 3,..."
...,...
47954,"[16, 16, 16, 6, 14, 16, 16, 16, 16, 16, 16, 16..."
47955,"[16, 7, 16, 3, 16, 16, 16, 16, 16, 16, 16, 16,..."
47956,"[16, 2, 16, 16, 7, 15, 16, 16, 16, 16, 16, 16,..."
47957,"[16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 1..."


In [225]:
df['pos_label_ids']

,pos_label_ids
0,"[19, 10, 19, 35, 34, 10, 17, 29, 31, 7, 16, 10..."
1,"[11, 19, 35, 22, 35, 29, 31, 16, 29, 11, 11, 1..."
2,"[16, 19, 17, 32, 11, 19, 10, 7, 17, 11, 16, 1,..."
3,"[22, 32, 10, 7, 16, 11, 16, 10, 16, 19, 2, 16,..."
4,"[17, 16, 16, 17, 17, 32, 17, 1, 17, 1, 11, 5, ..."
...,...
47954,"[17, 16, 17, 17, 17, 36, 34, 22, 36, 29, 31, 5..."
47955,"[10, 17, 1, 11, 16, 19, 34, 7, 16, 10, 7, 11, ..."
47956,"[33, 17, 21, 11, 17, 6, 19, 1, 19, 19, 32, 19,..."
47957,"[10, 24, 1, 19, 35, 34, 11, 19, 10, 7, 34, 5, ..."


In [221]:
# Train teste bolek demeli hem POS ucun Hem de NER ucun ayri ayri output olmalidi multi Task ucun ona gore y hissesini y_ner ve
# y_pos kimi ayirdiqdan sonra train_test_split istifade edecem

In [226]:
from sklearn.model_selection import train_test_split
X = np.array(df['input_ids'].to_list(),dtype=np.int32)
y_ner = np.array(df['label_ids'].to_list(),dtype=np.int32)
y_pos=np.array(df['pos_label_ids'].to_list(), dtype=np.int32)

# Pozitional ID-lər (0, 1, 2, ..., MAX_LEN-1)
pos_ids = np.tile(np.arange(X.shape[1]), (X.shape[0], 1))

X_train, X_test, y_ner_train, y_ner_test, y_pos_train, y_pos_test, pos_train, pos_test = train_test_split(
    X, y_ner, y_pos, pos_ids, test_size=0.2, random_state=42
)

In [227]:
X.shape[0], y_ner.shape[0], y_pos.shape[0], pos_ids.shape[0]

(47959, 47959, 47959, 47959)

In [228]:
print("X dtype:", X.dtype)

X dtype: int32


In [229]:
# Tensorflow datasina cevirek

In [230]:
batch_size=32

train_dataset = tf.data.Dataset.from_tensor_slices(
    ((X_train, pos_train), (y_ner_train, y_pos_train))
).batch(batch_size).shuffle(1000).prefetch(tf.data.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_tensor_slices(
    ((X_test, pos_test), (y_ner_test, y_pos_test))
).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [231]:
train_dataset

<_PrefetchDataset element_spec=((TensorSpec(shape=(None, 35), dtype=tf.int32, name=None), TensorSpec(shape=(None, 35), dtype=tf.int64, name=None)), (TensorSpec(shape=(None, 35), dtype=tf.int32, name=None), TensorSpec(shape=(None, 35), dtype=tf.int32, name=None)))>

In [232]:
valid_dataset

<_PrefetchDataset element_spec=((TensorSpec(shape=(None, 35), dtype=tf.int32, name=None), TensorSpec(shape=(None, 35), dtype=tf.int64, name=None)), (TensorSpec(shape=(None, 35), dtype=tf.int32, name=None), TensorSpec(shape=(None, 35), dtype=tf.int32, name=None)))>

In [237]:
#Encoder base transformer model

encoder_inputs=tf.keras.Input(shape=[None,], name='encoder_inputs')

# encoder ama bu defe POS ucu
pos_encoder_inputs=tf.keras.Input(shape=[None,], name='pos_encoder_inputs')

#Embedding layer
embed_layer=tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=128, mask_zero=True, name='embed_layer')

# Same Pos ucun emdedding layer
pos_em_lay=tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=128,mask_zero=True, name='pos_em_layer')

#encoder embedding
encoder_embed=embed_layer(encoder_inputs)
pos_encoder_embed=pos_em_lay(pos_encoder_inputs)
sum_pos_and_input_embed=encoder_embed+pos_encoder_embed


#Positional embed layer
embed_size=128
pos_embed_layer=tf.keras.layers.Embedding(MAX_LEN, embed_size)

pos_encoder=tf.keras.layers.Lambda(lambda x: tf.range(start=0,limit=tf.shape(x)[1],
                                                            delta=1))(encoder_inputs)

positional_embedding=pos_embed_layer(pos_encoder)




#concat tokrn and positional embedding
encoder_embedding=positional_embedding+sum_pos_and_input_embed

#Add multihead attentioan layer

num_heads=3
encoder_attention=tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size)(encoder_embedding,encoder_embedding)
encoder_attention=tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_attention+encoder_embedding)


# add Feed Forward (Dense) layer
ff_dim=512
encoder_ff=tf.keras.layers.Dense(ff_dim, activation='relu')(encoder_attention)
encoder_ff=tf.keras.layers.Dense(embed_size)(encoder_ff)
encoder_ff=tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoder_ff+encoder_attention)


#Add output layer Multi task olduguna gore hem Ner ucun Hem POS ucun ayri ayri output yazmaliyiq
ner_output=tf.keras.layers.Dense(len(tag2id), activation='softmax', name='ner_output')(encoder_ff)

pos_output=tf.keras.layers.Dense(len(pos2id), activation='softmax', name='pos_output')(encoder_ff)

In [236]:
#Callback and learning schedule
learning_rate_schedule=tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=1000,
    decay_rate=0.9
)

early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [238]:
#Make model
model=tf.keras.Model(inputs=[encoder_inputs,pos_encoder_inputs], outputs=[ner_output, pos_output])

In [239]:
#optimizer
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)

In [240]:
#model compiling
model.compile(optimizer=optimizer,

              loss= {'ner_output': 'sparse_categorical_crossentropy',
                    'pos_output': 'sparse_categorical_crossentropy' },

               metrics={
        'ner_output': 'accuracy',
        'pos_output': 'accuracy'}
    )

In [241]:
#model fitting
model.fit(train_dataset, validation_data=valid_dataset, epochs=10)

Epoch 1/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - loss: 0.5653 - ner_output_accuracy: 0.9485 - ner_output_loss: 0.2162 - pos_output_accuracy: 0.9045 - pos_output_loss: 0.3491 - val_loss: 0.1913 - val_ner_output_accuracy: 0.9692 - val_ner_output_loss: 0.1023 - val_pos_output_accuracy: 0.9703 - val_pos_output_loss: 0.0889
Epoch 2/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.1417 - ner_output_accuracy: 0.9748 - ner_output_loss: 0.0798 - pos_output_accuracy: 0.9776 - pos_output_loss: 0.0619 - val_loss: 0.1705 - val_ner_output_accuracy: 0.9722 - val_ner_output_loss: 0.0920 - val_pos_output_accuracy: 0.9745 - val_pos_output_loss: 0.0785
Epoch 3/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.1027 - ner_output_accuracy: 0.9802 - ner_output_loss: 0.0596 - pos_output_accuracy: 0.9842 - pos_output_loss: 0.0432 - val_loss: 0.1607 - val_ner_output_accuracy: 0.9740 - val_ner_output_loss: 0.0885 - val_pos_output_accuracy: 0.9772 - val_pos_output_loss: 0.0723
Epoch 4/10
11

In [247]:
# Proqnozları al
pred_ner_probs, pred_pos_probs = model.predict(valid_dataset)

# Argmax ilə ən yüksək ehtimallı etiketləri seç
pred_ner_labels = np.argmax(pred_ner_probs, axis=-1)
pred_pos_labels = np.argmax(pred_pos_probs, axis=-1)

# İlk 2 nümunəni göstər
for i in range(2):
    print(f"\nExample {i+1}")
    print(f"{'Token':15} {'True NER':10} {'Pred NER':10} {'True POS':10} {'Pred POS':10}")
    print("-" * 60)

    input_ids = X_test[i]
    true_ner_ids = y_ner_test[i]
    pred_ner_ids = pred_ner_labels[i]
    true_pos_ids = y_pos_test[i]
    pred_pos_ids = pred_pos_labels[i]

    for token_id, true_ner_id, pred_ner_id, true_pos_id, pred_pos_id in zip(input_ids, true_ner_ids, pred_ner_ids, true_pos_ids, pred_pos_ids):
        if token_id != 0:  # padding tokenləri atla
            token = [k for k, v in vocab.items() if v == token_id][0]
            true_ner_tag = id2tag[true_ner_id]
            pred_ner_tag = id2tag[pred_ner_id]
            true_pos_tag = id2pos[true_pos_id]
            pred_pos_tag = id2pos[pred_pos_id]

            print(f"{token:15} {true_ner_tag:10} {pred_ner_tag:10} {true_pos_tag:10} {pred_pos_tag:10}")


300/300 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

Example 1
Token           True NER   Pred NER   True POS   Pred POS  
------------------------------------------------------------
The             O          O          DT         DT        
report          O          O          NN         NN        
calls           O          O          VBZ        VBZ       
on              O          O          IN         IN        
President       B-per      B-per      NNP        NNP       
Bush            I-per      I-per      NNP        NNP       
and             O          O          CC         CC        
Congress        B-org      B-org      NNP        NNP       
to              O          O          TO         TO        
urge            O          O          VB         VB        
Chinese         B-gpe      B-gpe      JJ         JJ        
officials       O          O          NNS        NNS       
not             O          O          RB         RB        
to              O          O          TO       

In [246]:
def predict_sentence(sentence, model, vocab, id2tag, id2pos, MAX_LEN):
    # 1. Tokenləşdir (sadə boşluqla ayırma)
    tokens = sentence.split()

    # 2. Tokenləri id-ə çevir
    token_ids = [vocab.get(token, 0) for token in tokens]

    # 3. Padding et
    input_ids = tf.keras.preprocessing.sequence.pad_sequences([token_ids], maxlen=MAX_LEN, padding='post', value=0)

    # 4. Pozisional id hazırla
    pos_ids = np.tile(np.arange(MAX_LEN), (1, 1))

    # 5. Modeldən proqnoz al
    pred_ner_probs, pred_pos_probs = model.predict([input_ids, pos_ids])

    # 6. Argmax et
    pred_ner = np.argmax(pred_ner_probs, axis=-1)[0]
    pred_pos = np.argmax(pred_pos_probs, axis=-1)[0]

    # 7. Nəticələri çap et
    print(f"{'Token':15} {'Predicted NER':15} {'Predicted POS'}")
    print("-" * 45)
    for i, token in enumerate(tokens):
        print(f"{token:15} {id2tag.get(pred_ner[i], 'O'):15} {id2pos.get(pred_pos[i], 'NN')}")

# Nümunə istifadə:
sentence = "Gloria Steinem works with the National Organization for Women to promote feminism."
predict_sentence(sentence, model, vocab, id2tag, id2pos, MAX_LEN)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Token           Predicted NER   Predicted POS
---------------------------------------------
Gloria          B-per           NNP
Steinem         O               NN
works           O               VBZ
with            O               IN
the             O               DT
National        B-org           NNP
Organization    I-org           NNP
for             I-org           IN
Women           I-org           NNP
to              O               TO
promote         O               VB
feminism.       O               NN
